In [1]:
#IMPORTING NEEDED LIBRARIES AND APIs

import json
import pandas as pd
import urllib3
import re
import gspread
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
import requests
from unidecode import unidecode
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

with open(r'./creds.json') as f:
  data = json.load(f)

In [2]:
#CONFIGURING GOOGLE SHEETS API
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

client = gspread.authorize(creds)

sheet1 = client.open('sheets_database').worksheet('Sheet1')
data_1 = sheet1.get_all_records()
data_1x = pd.DataFrame(data_1)

In [2]:
#CONFIGURING FB BUSINESS API
my_app_id = 'xxx'
my_app_secret = 'xxx'
my_access_token = 'xxx'
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)
my_account = AdAccount('xxx')
ads = my_account.get_ads()

In [4]:
#SETTING FB QUERY FOR DF
fields = [ 'ad_name', 'spend', 'impressions', 'inline_link_clicks']
params = { 'level': 'ad', 'filtering': [], 'breakdowns': ['region'], 'time_increment':'1', 'time_range': {'since':'2020-06-29','until':'2020-07-05'}, } 
query_fb = pd.DataFrame(my_account.get_insights(fields=fields, params=params))

In [5]:
#REPLACING AND NORMALIZING VALUES IN FB DB
ss = query_fb['region'].str.replace("(", '')
ss2 = ss.str.replace(')', '')
ss3 = ss2.str.replace(' state', '')
ss3_df = pd.DataFrame(ss3)
res = ss3_df['region'].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')
ss_fb = pd.DataFrame(res)
query_fb['region'] = ss_fb

In [88]:
#IMPORTING ALL NEEDED TABLES
db_inapp = pd.read_excel(r'xxx')
db_pedidos = pd.read_excel(r'xxx')
db_install = pd.read_excel(r'xxx', sheet_name='dbase')
db_fb = query_fb
db_gads = data_1x
db_vtrack = pd.read_excel('xxx', sheet_name='dbase')
db_atribuicao = pd.read_excel('xxx', sheet_name='dbase')
db_teste = pd.read_excel('xxx')

In [ ]:
#RENAMING AND CORRECTING DB INTEGER GLITCH
db_teste = db_teste.rename(columns={"Cidade (local geográfico)": "Cidade"})
db_pedidos['Complemento'] = 1
db_install.rename(columns={'Hora Install': 'Hora'})

In [90]:
#NORMALIZING REGIONAL PARAMETERS
ss = db_fb['region'].str.replace("(", '')
ss2 = ss.str.replace(')', '')
ss3 = ss2.str.replace(' state', '')
ss3_df = pd.DataFrame(ss3)
res = ss3_df['region'].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')
ss_fb = pd.DataFrame(res)
db_fb['region'] = ss_fb

In [91]:
#2
res_ped = db_pedidos['Cidade'].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')
ss_pedidos = pd.DataFrame(res_ped)
db_pedidos['Cidade'] = ss_pedidos

In [92]:
#3
res_teste = db_teste['Cidade'].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')
ss_teste = pd.DataFrame(res_teste)
db_teste['Cidade'] = ss_teste

In [93]:
#USING OHE TO TRANSFORM CATEGORICAL INTO NUMERICAL FEATURES
X = db_inapp.iloc[:,2].values
re = OneHotEncoder()
jaxle = X.reshape(-1, 1)
jaxre = re.fit_transform(jaxle).toarray()
bobby = pd.DataFrame(jaxre)
bobby.columns = re.get_feature_names()

In [94]:
#CORRECTING COLUMN NAMES
def drop_prefix(self, prefix):
    self.columns = self.columns.str.lstrip(prefix)
    return bobby

drop_prefix(bobby, 'x0_')

bobby = bobby.fillna(0)

In [95]:
#IMPUTING NEW COLUMNS TO DF
db_inapp['column1'] = bobby['column1']
db_inapp['column2'] = bobby['column2']
db_inapp['column3'] = bobby['column3']
db_inapp['column4'] = bobby['column4']
db_inapp['column5'] = bobby['column5']

In [112]:
#CREATING NEW DF ONLY WITH UNIQUE VALUES FROM COLUMN
db_inapp_unique = db_inapp.groupby('col_id').head(1)

In [107]:
#IMPORTING SQLITE AND CREATING DB TABLES
import sqlite3

dbfile = sqlite3.connect(r'C:\db.db')
cur = dbfile.cursor()

cur.execute("CREATE TABLE 'Jornada_App' ('Data Event', 'Event Name', 'Evento', 'Data Install', 'Media Source', 'Channel', 'Campaign', 'City', 'State', 'Postal Code', 'AppsFlyer ID', 'Platform', 'Origem', 'Clique', 'Adicionou ao carrinho', 'Cadastro', 'Formas de Pagamento', 'Pedido');")
cur.execute("CREATE TABLE 'Pedidos App' ('Pesquisa', 'Pedido', 'Cliente', 'Telefone', 'ID do Cliente', 'E-mail', 'Data', 'Data Ajustada', 'Hora', 'CNPJ', 'Revenda', 'Endereço', 'Nº', 'Complemento', 'Bairro', 'Referência', 'CEP', 'Cidade', 'UF', 'Canal Hub', 'Origem Acesso', 'Modo', 'Status', 'Data de Entrega', 'Produto', 'Preço', 'Quantidade', 'Vasilhame', 'Preço Vasilhame', 'Forma Pagto', 'Valor Pedido', 'Origem do Cliente', 'Mídia');")
cur.execute("CREATE TABLE Instalações ('Data Install', 'Hora Install', Midia, Channel, Campanha, Cidade, Estado, CEP, ID, Instalação, Platform, 'Device Type', Hora);")
cur.execute("CREATE TABLE FB ('ad_name', 'date_start', 'date_stop', 'impressions','inline_link_clicks', 'spend');")
cur.execute("CREATE TABLE 'Google Ads' ('Date', 'City', 'Campaign name', 'Ad group name', 'Impressions', 'Clicks', 'Cost', 'Conversions');")
cur.execute("CREATE TABLE 'Video Track' (Data, Secundagem, Programa, 'Hora da Inserção', Mercado, Veículo, Inserção, Categoria, Atribuição, Investimento);")
cur.execute("CREATE TABLE 'Atribuição' ('Data', 'Secundagem', 'Programa', 'Hora da Inserção', 'Cidade','Emissora', 'Janela Atribuição', 'Pedidos Atribuídos', 'Instalações Atribuídas', 'Investimento');")
cur.execute("CREATE TABLE 'GAds Provisoria' ('Data', 'Grupo de anúncios', 'Cidade', 'Moeda', 'Cliques', 'Impressões','Custo');")
cur.execute("CREATE TABLE 'Jornada_App_Unique' ('Data Event', 'Event Name', 'Evento', 'Data Install', 'Media Source', 'Channel', 'Campaign', 'City', 'State', 'Postal Code', 'AppsFlyer ID', 'Platform', 'Origem', 'Clique', 'Adicionou ao carrinho', 'Cadastro', 'Formas de Pagamento', 'Pedido');")


dbfile.commit()

In [108]:
#DEFINING TABLE NAMES AND APPEND FUNCTION
table_name1 = 'db1'
table_name2 = 'db2'
table_name3 = 'db3'
table_name4 = 'db4'
table_name5 = 'db5'
table_name6 = 'db6'
table_name7 = 'db7'
table_name8 = 'db8'
table_name9 = 'db9'


def append_tables(): 
    
    db_inapp.to_sql(table_name1, dbfile, if_exists='append', index=False),
    db_pedidos.to_sql(table_name2, dbfile, if_exists='append', index=False, chunksize=1000),
    db_install.to_sql(table_name3, dbfile, if_exists='append', index=False),
    db_fb.to_sql(table_name4, dbfile, if_exists='append', index=False),
    db_gads.to_sql(table_name5, dbfile, if_exists='append', index=False),
    db_vtrack.to_sql(table_name6, dbfile, if_exists='append', index=False),
    db_atribuicao.to_sql(table_name7, dbfile, if_exists='append', index=False),
    db_teste.to_sql(table_name8, dbfile, if_exists='append', index=False)
    db_inapp_unique.to_sql(table_name9, dbfile, if_exists='append', index=False)

In [109]:
#DEFINING FUNCTION TO DELETE NULL ROWS
def delete_nulls():
                    
    cur.execute("DELETE FROM db1 WHERE db1.Data='';"),
    cur.execute("DELETE FROM db2 WHERE db2.Data='';"),
    cur.execute("DELETE FROM db3 WHERE db3.Data='';"),
    cur.execute("DELETE FROM db4 WHERE db4.Data='';"),
    cur.execute("DELETE FROM db5 WHERE db5.Data='';"),
    cur.execute("DELETE FROM db6 WHERE db6.Data='';"),
    cur.execute("DELETE FROM db7 WHERE db7.Data='';")
    cur.execute("DELETE FROM db8 WHERE db8.Data='';")
    cur.execute("DELETE FROM db9 WHERE db9.Data='';")
    dbfile.commit()

In [110]:
#DEFINING FUNCTION TO CORRECT FLOAT READING
def bases_commas():
    
    cur.execute("""
    UPDATE 
        db5
    SET
        spend = REPLACE(spend, '.',',')
    WHERE
        spend IS NOT NULL""")
    
    cur.execute("""
    UPDATE 
        db6
    SET
        Preço = REPLACE(Preço, '.',',')
    WHERE
        Preço IS NOT NULL;""")

In [111]:
#EXECUTING FUNCTIONS ALTOGHETER
bases_commas()
append_tables()
delete_nulls()

In [21]:
#TRANSFORMING PYTHON NOTEBOOK INTO .PY SCRIPT
!jupyter nbconvert --to script sqldb_setup_v1.ipynb

[NbConvertApp] Converting notebook ultragaz_sqlSheets_setup.ipynb to script
[NbConvertApp] Writing 4316 bytes to ultragaz_sqlSheets_setup.py
